In [2]:
from functions import *

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [11]:
from functions import *
pcd_path = Path("../modular_polygon_generation/libcore/data/maps/area_1.pcd")
chen_graph_path_input = Path("../output/tests/area_1_parameters/sampling_shape/run_1762356356")
vox_path = Path("../data/voxel_maps/area_1/")
pcd = load_point_cloud(pcd_path)
# Load chen skeleton graph
chen_skeleton_graph, id_map_old = load_graph(chen_graph_path_input / "node_list.txt", chen_graph_path_input / "edge_list.txt")
id_map = np.load(chen_graph_path_input / "chen_voxel_id_map.npy")
id_map_full = np.load(chen_graph_path_input / "chen_id_map_full.npy")
dist_map = np.load(chen_graph_path_input / "chen_dist_map.npy")
M = np.load(vox_path / "area_1_size_0_1_M.npy")

visualize([VisObjGraph(chen_skeleton_graph), VisObjPCD(pcd), VisObjVoxel(id_map, M)])

voxel_grid_flooded= np.load(vox_path / "area_1_size_0_1_voxel_grid_flooded.npy")
visualize([VisObjVoxel(id_map_full, M)])
visualize([VisObjVoxel(dist_map, M)])

Loading point cloud from ../modular_polygon_generation/libcore/data/maps/area_1.pcd...
Loaded point cloud from ../modular_polygon_generation/libcore/data/maps/area_1.pcd with 44026810 points.
Loading graph from ../output/tests/area_1_parameters/sampling_shape/run_1762356356/node_list.txt and ../output/tests/area_1_parameters/sampling_shape/run_1762356356/edge_list.txt...
Loaded graph with 1699 nodes and 1834 edges.


In [13]:
from skimage.draw import line_nd
# inverse transformation matrix
M_inv = np.linalg.inv(transformation_matrix)

outlaiers = set()
for edge in chen_skeleton_graph.edges:
    n1 = chen_skeleton_graph.nodes[edge[0]]
    n2 = chen_skeleton_graph.nodes[edge[1]]
    coords_1 = n1['coords']
    coords_2 = n2['coords']
    coords_1_h = np.append(coords_1, 1)
    coords_2_h = np.append(coords_2, 1)
    v_pos_1 = (M_inv @ coords_1_h)[:3]
    v_pos_2 = (M_inv @ coords_2_h)[:3]
    # compute line voxels between v_pos_1 and v_pos_2
    line_voxels = np.array(line_nd(tuple(v_pos_1.astype(int)), tuple(v_pos_2.astype(int)))).T[1:] # skip the first entry, hence its the node itself
    for voxel in line_voxels:
        if voxel_grid[tuple(voxel)] == 1:
            outlaiers.add(edge)
print(f"Total outlier nodes: {len(outlaiers)}")

Total outlier nodes: 10


In [13]:
from functions import *
pcd_path = Path("../modular_polygon_generation/libcore/data/maps/area_1_downsampled_0_1.pcd")
pcd = load_point_cloud(pcd_path)
bounding_box = np.array([[-26.0, -8.0, -1.0], [8.0, 50.0, 3.3]])
start_pt = np.array([-5.0, 9.0, 1.1])
visualize(VisObjPCD(pcd, bounding_box=bounding_box, start_pt=start_pt))

Loading point cloud from ../modular_polygon_generation/libcore/data/maps/area_1_downsampled_0_1.pcd...
Loaded point cloud from ../modular_polygon_generation/libcore/data/maps/area_1_downsampled_0_1.pcd with 495103 points.
Adding bounding box to visualization...
Bounding box min: [-26.  -8.  -1.], max: [ 8.  50.   3.3]
Adding start point to visualization...


In [12]:
from functions import *
import numpy as np

pcd_path = Path("../modular_polygon_generation/libcore/data/maps/area_1_downsampled_0_1.pcd")
pcd = load_point_cloud(pcd_path)
bounding_box = np.array([[-26.0, -8.0, -1.0], [8.0, 50.0, 3.3]])

voxel_grid_inv = np.load("../data/voxel_maps/area_1/area_1_size_0_1_voxel_grid_inverted.npy")
# generate 100 random points within the voxel grid where voxel_grid_inv == 1
indices = np.array(np.where(voxel_grid_inv == 1)).T
random_indices = indices[np.random.choice(indices.shape[0], 200, replace=False)]
# convert voxel indices to world coordinates
M = np.load("../data/voxel_maps/area_1/area_1_size_0_1_M.npy")
idx_h = np.hstack((random_indices, np.ones((random_indices.shape[0], 1))))
world_coords = (idx_h @ M.T)[:, :3]
voxel_size = M[0,0]  # assuming uniform scaling
world_coords_center = world_coords + voxel_size / 2.0

point_pairs = world_coords_center.reshape(-1, 2, 3)
point_pairs = point_pairs.reshape(-1, 6)

for pair in point_pairs:
    p1 = pair[:3]
    p2 = pair[3:]
    print(f"Point 1: {p1}, Point 2: {p2}")
    print(f"distance: {np.linalg.norm(p2 - p1)}")
    #visualize(VisObjPCD(pcd, bounding_box=bounding_box, start_pt=p1, end_pt=p2))

np.save("point_pairs.npy", point_pairs)

Loading point cloud from ../modular_polygon_generation/libcore/data/maps/area_1_downsampled_0_1.pcd...
Loaded point cloud from ../modular_polygon_generation/libcore/data/maps/area_1_downsampled_0_1.pcd with 495103 points.
Point 1: [-3.72299919 22.08299999  1.727     ], Point 2: [-20.42299919  18.08299999   2.727     ]
distance: 17.201453426963667
Point 1: [-14.52299919  -1.61700001   1.627     ], Point 2: [ 0.67700081 28.48299999  3.027     ]
distance: 33.74922221325997
Point 1: [-9.22299919  5.48299999  0.927     ], Point 2: [ 1.27700081 15.48299999  2.727     ]
distance: 14.611296999240007
Point 1: [0.77700081 7.28299999 0.227     ], Point 2: [-17.72299919  32.88299999   1.227     ]
distance: 31.600791129337253
Point 1: [-1.32299919 16.98299999  1.527     ], Point 2: [ 0.97700081 40.18299999  1.027     ]
distance: 23.319090891370532
Point 1: [-13.42299919  14.28299999   2.627     ], Point 2: [-6.02299919 31.68299999  0.327     ]
distance: 19.047572023751478
Point 1: [-9.92299919  3.6

In [8]:
point_pairs.shape

(100, 6)